In [ ]:
!pip install pandas
!pip install requests
!pip install bs4

In [33]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from requests.exceptions import ChunkedEncodingError
import re
import os
from deep_translator import GoogleTranslator
import glob

In [ ]:
url = "https://ideasai.com/"  # Sustituye esto por la URL de la página que quieres scrapear
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')


In [ ]:
# Ahora puedes usar 'soup' para encontrar elementos en la página. Por ejemplo:
titulo = soup.find('title').text # Esto obtendrá el título de la página
titulo

In [ ]:
# All .tag-link inside <div class="tag-container">
tags = soup.find('div', attrs={'class': 'tag-container'}).find_all('a', attrs={'class': 'tag-link'})

tags_list = []

for tag in tags:
    tags_list.append(tag.text)
    
    num_tags = len(tags_list)
    
print(f"El número de etiquetas es: {num_tags}")

In [ ]:
# Enter in one of the tags and get h3 classname .idea
tag = tags_list[0]
tag_url = url + tag
tag_url

response = requests.get(tag_url)
soup = BeautifulSoup(response.text, 'html.parser')

ideas = soup.find_all('h3', attrs={'class': 'idea'})

ideas_list = []

for idea in ideas:
    ideas_list.append(idea.text)
    num_ideas = len(ideas_list)

print(f"El número de ideas es: {num_ideas}")

In [ ]:
# create an empty list to store the data
data = []

# iterate over all the tags
for i, tag in enumerate(tags_list, 1):
    try:
        tag_url = url + tag
        response = requests.get(tag_url)
        response.raise_for_status()  # verifica si hay errores en la respuesta HTTP
        soup = BeautifulSoup(response.text, 'html.parser')
        ideas = soup.find_all('h3', attrs={'class': 'idea'})
        for idea in ideas:
            data.append({'Tag': tag, 'Idea': idea.text})
        print(f"Enlaces procesados: {i}, Enlaces restantes: {len(tags_list) - i}")
    except ChunkedEncodingError as e:
        print(f"Error: {e}. Reintentando...")

# create a dataframe from the list
df = pd.DataFrame(data)

# save the results into a CSV file
df.to_csv('ideas.csv', index=False)

In [ ]:
# Cargar el archivo CSV en un DataFrame
df = pd.read_csv('ideas.csv')

# Limpiar la columna 'Idea' usando expresiones regulares
df['Tag'] = df['Tag'].apply(lambda x: re.sub(r'\s+', ' ', x).strip())

# Guardar el DataFrame modificado en un nuevo archivo CSV
df.to_csv('ideas_limpias.csv', index=False)

In [ ]:
# Cargar el archivo CSV modificado en un DataFrame
df_modificado = pd.read_csv('ideas_limpias.csv')

# Imprimir el DataFrame
df_modificado.head()

In [ ]:
# Load the modified CSV file into a DataFrame
df = pd.read_csv('ideas_limpias.csv')

# Translate the 'Idea' column from English to Spanish using GoogleTranslator
df['ideas'] = df['Idea'].apply(lambda x: GoogleTranslator(source='en', target='es').translate(x))

# Drop the original 'Idea' column
df.drop('Idea', axis=1, inplace=True)

# Save the translated DataFrame to a new CSV file
df.to_csv('ideas_traducidas.csv', index=False)

# Load the translated CSV file into a DataFrame
df.head()


In [ ]:
# Cargar el archivo CSV en un DataFrame
df = pd.read_csv('ideas_limpias.csv')

# Cargar el archivo CSV en un DataFrame
df = pd.read_csv('ideas_limpias.csv')

print(f"El número de ideas es: {df.shape[0]}")

# Check in csv what are the most repeated tags
repeated = df['Tag'].value_counts()

print(f"El número de etiquetas es: {repeated.shape[0]}")


In [ ]:
# Cargar el archivo CSV en un DataFrame
df = pd.read_csv('ideas_limpias.csv')

# Crear la carpeta si no existe
carpeta_export = 'csv-export'
os.makedirs(carpeta_export, exist_ok=True)

# Dividir el DataFrame por etiquetas y guardar cada subconjunto en un archivo en la carpeta
for etiqueta, subconjunto in df.groupby('Tag'):
    # Crear el nombre del archivo y la ruta en la carpeta
    nombre_archivo = f'{etiqueta}_subconjunto.csv'
    ruta_archivo = os.path.join(carpeta_export, nombre_archivo)
    
    # Guardar el subconjunto en el archivo en la carpeta
    subconjunto.to_csv(ruta_archivo, index=False)

    print(f'Se ha creado el archivo {ruta_archivo} con {len(subconjunto)} filas.')


In [35]:
# Carpeta de exportación para JSON
carpeta_csv = 'csv-export'
carpeta_export = 'json-export'
os.makedirs(carpeta_export, exist_ok=True)

# Obtener la lista de archivos CSV en la carpeta csv-export
archivos_csv = glob.glob(os.path.join(carpeta_csv, '*.csv'))

# Iterar sobre los archivos CSV
for archivo_csv in archivos_csv:
    # Leer el archivo CSV en un DataFrame
    df = pd.read_csv(archivo_csv)

    # Crear el nombre del archivo JSON
    nombre_json = os.path.splitext(os.path.basename(archivo_csv))[0] + '.json'
    ruta_json = os.path.join(carpeta_export, nombre_json)

    # Guardar el DataFrame como JSON en la carpeta de exportación
    df.to_json(ruta_json, orient='records', lines=True)

    print(f'Se ha creado el archivo {ruta_json} a partir de {archivo_csv}.')

Se ha creado el archivo json-export\3d Startup Ideas_subconjunto.json a partir de csv-export\3d Startup Ideas_subconjunto.csv.
Se ha creado el archivo json-export\Ability Startup Ideas_subconjunto.json a partir de csv-export\Ability Startup Ideas_subconjunto.csv.
Se ha creado el archivo json-export\Able Startup Ideas_subconjunto.json a partir de csv-export\Able Startup Ideas_subconjunto.csv.
Se ha creado el archivo json-export\Abroad Startup Ideas_subconjunto.json a partir de csv-export\Abroad Startup Ideas_subconjunto.csv.
Se ha creado el archivo json-export\Accelerator Startup Ideas_subconjunto.json a partir de csv-export\Accelerator Startup Ideas_subconjunto.csv.
Se ha creado el archivo json-export\Accept Startup Ideas_subconjunto.json a partir de csv-export\Accept Startup Ideas_subconjunto.csv.
Se ha creado el archivo json-export\Accepted Startup Ideas_subconjunto.json a partir de csv-export\Accepted Startup Ideas_subconjunto.csv.
Se ha creado el archivo json-export\Accessed Startu